In [14]:
from mistralai import Mistral, DocumentURLChunk
from mistralai.models import OCRResponse
from pathlib import Path

In [ ]:
client = Mistral(api_key="YOUR-KEY")

In [ ]:
def transcribe(pdf_path: Path, output_path: Path):
    with open(pdf_path, "rb") as f:
        pdf = f.read()
    uploaded = client.files.upload(
        file={
            "file_name": pdf_path.name,
            "content": pdf,
        },
        purpose="ocr",
    )
    url = client.files.get_signed_url(file_id=uploaded.id, expiry=1)
    response = client.ocr.process(
        document=DocumentURLChunk(document_url=url.url),
        model="mistral-ocr-latest",
        include_image_base64=False,
    )
    md = "\n\n".join([page.markdown for page in response.pages])
    with open(output_path, "w", encoding="utf-8") as f:
        f.write(md)

In [ ]:
pdf_path = Path("ensayo.pdf")
try:
    transcribe(pdf_path, Path("index.md"))
except Exception as e:
    print(f"{pdf_path.name}: {e}")